# 用單個字進行新聞標題預測

參考網路上資料：https://juejin.im/entry/5a44bc4c6fb9a0451f313f2f?fbclid=IwAR2k7F9mi1vkJ5PGSiXKvODnmmhXv3NKxRwnNk3jG_qGgQTNT_lSPk93sik

## 神經網路架構

1. 將 6513 維的文字壓到 256 維
2. 用 128 個神經元的 LSTM 做隱藏層
3. 加一層 dropout 避免 over-fitting
4. 再加一層 8 個神經元的隱藏層
5. 因為 y 是 one-hot encoding，所以 Activation Function 是 softmax
6. 用 categorical_crossentropy 做 loss function, optimizer 用 Adam

### 輸入：新聞標題(拆成單個字，並轉換成ID)
### 輸出：標題類別編號

## 1. 資料預處理

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

讀入5份爬蟲的新聞標題資料並接合

In [2]:
df1 = pd.read_excel('總資料整理1.xlsx')
df2 = pd.read_excel('總資料整理2.xlsx')
df3 = pd.read_excel('總資料整理3.xlsx')
df4 = pd.read_excel('總資料整理4.xlsx')
df5 = pd.read_excel('總資料整理5.xlsx')

In [3]:
df = pd.concat([df1, df2, df3, df4, df5], axis = 0)

分析新聞標題類型

In [4]:
df.類別.unique()

array(['娛樂', '生活', '社會', '政治', '財經', '國際', '體育'], dtype=object)

讀取標題

In [5]:
A=[]
A=df['標題']

將每句標題中的特殊字元去除

In [7]:
import re

In [8]:
r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】〈〉～「」é；《》？“”‘’！[\\]^_`{|}~]+'
r2 = "[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+"
r3 =  "[.!//_,$&%^*()<>+\"'?@#-|:~{}]+|[——！\\\\，。=？、：“”‘’《》【】￥……（）]+"
r4 =  "\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：“”‘’￥……（）《》【】]"

In [9]:
A1=[]
for i in A:
    i=re.sub(r1,'',i)
    A1.append(i)

In [10]:
A2=[]
for i in A1:
    i=re.sub(r2,'',i)
    A2.append(i)

In [11]:
A3=[]
for i in A2:
    i=re.sub(r3,'',i)
    A3.append(i)

In [12]:
A4=[]
for i in A3:
    i=re.sub(r4,'',i)
    A4.append(i)

In [13]:
A4[0]

'原創鄧超彭于晏相約看球賽對鏡頭互做鬼臉二人彼此調侃友誼深厚'

新建一個新的DataFrame，只包含我們需要用到的資料 'label' 與 'title'

In [14]:
L = list(df['label'])

In [15]:
dfu = pd.DataFrame({'label' : L,
                                'title' : A4}, 
                                columns=['label','title'])

將title的每個字分開

In [16]:
dfu['words'] = dfu['title'].apply(lambda x: re.findall('[\x80-\xff]{3}|[\w\W]', x))

In [17]:
dfu.head()

,label,title,words
0,1,原創鄧超彭于晏相約看球賽對鏡頭互做鬼臉二人彼此調侃友誼深厚,"[原, 創, 鄧, 超, 彭, 于, 晏, 相, 約, 看, 球, 賽, 對, 鏡, 頭, ..."
1,1,孫東雲將註銷帳號月日入伍服兵役,"[孫, 東, 雲, 將, 註, 銷, 帳, 號, 月, 日, 入, 伍, 服, 兵, 役]"
2,1,王小帥朋友圈是什麼情況王小帥朋友圈發了什麼內容,"[王, 小, 帥, 朋, 友, 圈, 是, 什, 麼, 情, 況, 王, 小, 帥, 朋, ..."
3,1,楊明逸大雪焚心殺青最慘富二代為愛入局,"[楊, 明, 逸, 大, 雪, 焚, 心, 殺, 青, 最, 慘, 富, 二, 代, 為, ..."
4,1,樂基兒挺八月孕肚腳踩恨天高與閨蜜大跳熱舞組圖,"[樂, 基, 兒, 挺, 八, 月, 孕, 肚, 腳, 踩, 恨, 天, 高, 與, 閨, ..."


建立字典，裡面包含了words中出現的所有字，'0'是該字出現的次數欄，'id'是為該字建立的id欄

In [18]:
all_words = []
for w in dfu['words']:
    all_words.extend(w)
word_dict = pd.DataFrame(pd.Series(all_words).value_counts())
word_dict['id'] = list(range(1, len(word_dict)+1))

In [19]:
word_dict

,0,id
國,72763,1
人,69210,2
大,65927,3
的,58807,4
美,57337,5
中,53116,6
一,42670,7
不,42647,8
年,42202,9
日,35110,10


將words中的每個字映射到字典中對應的id

In [20]:
dfu['wordidlist'] = dfu['words'].apply(lambda x: list(word_dict['id'][x]))

In [21]:
dfu.head()

,label,title,words,wordidlist
0,1,原創鄧超彭于晏相約看球賽對鏡頭互做鬼臉二人彼此調侃友誼深厚,"[原, 創, 鄧, 超, 彭, 于, 晏, 相, 約, 看, 球, 賽, 對, 鏡, 頭, ...","[239, 178, 1142, 191, 1524, 2490, 2843, 253, 3..."
1,1,孫東雲將註銷帳號月日入伍服兵役,"[孫, 東, 雲, 將, 註, 銷, 帳, 號, 月, 日, 入, 伍, 服, 兵, 役]","[994, 170, 655, 47, 2453, 642, 1476, 646, 84, ..."
2,1,王小帥朋友圈是什麼情況王小帥朋友圈發了什麼內容,"[王, 小, 帥, 朋, 友, 圈, 是, 什, 麼, 情, 況, 王, 小, 帥, 朋, ...","[48, 27, 976, 1407, 113, 762, 26, 557, 294, 13..."
3,1,楊明逸大雪焚心殺青最慘富二代為愛入局,"[楊, 明, 逸, 大, 雪, 焚, 心, 殺, 青, 最, 慘, 富, 二, 代, 為, ...","[460, 92, 2537, 3, 704, 2391, 71, 299, 487, 28..."
4,1,樂基兒挺八月孕肚腳踩恨天高與閨蜜大跳熱舞組圖,"[樂, 基, 兒, 挺, 八, 月, 孕, 肚, 腳, 踩, 恨, 天, 高, 與, 閨, ...","[241, 270, 175, 828, 479, 84, 979, 1765, 1093,..."


找尋長度最長的字串

In [22]:
maxn = 0

for i in dfu['wordidlist']:
    if len(i) > maxn:
        maxn = len(i)

print(maxn)

56


## 2. 建立LSTM模型

In [23]:
from keras.preprocessing import sequence

Using TensorFlow backend.


將每個wordidlist padding成長度為25

In [24]:
maxlen = 25
dfu['wordidlist'] = list(sequence.pad_sequences(dfu['wordidlist'], maxlen=maxlen))

In [25]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras.optimizers

In [26]:
from sklearn.model_selection import train_test_split

將Input 與 Output 轉為 numpy array

In [27]:
X = np.array(list(dfu['wordidlist']))
Y = np.array(list(dfu['label']))

In [28]:
Y[1]

1

分離訓練與測試資料，並將Output 轉為 One-hot encoding的形式

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, random_state = 508)

In [37]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [38]:
y_train.shape

(341292, 8)

In [39]:
from keras.layers import Embedding, LSTM, Dropout

下面是訓練的神經網路模型，用dropout是為了避免over-fitting

In [40]:
model = Sequential()
model.add(Embedding(len(word_dict)+1, 256))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 256)         1667584   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
_________________________________________________________________
activation_2 (Activation)    (None, 8)                 0         
Total params: 1,865,736
Trainable params: 1,865,736
Non-trainable params: 0
_________________________________________________________________


訓練5個epoch，正確率約為82%

In [41]:
model.fit(x_train, y_train, batch_size=1000, epochs=5)

Epoch 1/5
341292/341292 [==============================] - 57s 168us/step - loss: 0.7859 - acc: 0.7178
Epoch 2/5
341292/341292 [==============================] - 57s 166us/step - loss: 0.5643 - acc: 0.7966
Epoch 3/5
341292/341292 [==============================] - 56s 165us/step - loss: 0.5320 - acc: 0.8060
Epoch 4/5
341292/341292 [==============================] - 56s 165us/step - loss: 0.5095 - acc: 0.8137
Epoch 5/5
341292/341292 [==============================] - 56s 165us/step - loss: 0.4900 - acc: 0.8196


In [42]:
score = model.evaluate(x_test, y_test)

146269/146269 [==============================] - 48s 328us/step


測試資料的正確率也落在80%左右

In [43]:
print('loss:', score[0])
print('正確率', score[1])

loss: 0.533040630892
正確率 0.805194538834


## 3. 將結果進行動態呈現

In [44]:
from ipywidgets import interact_manual

In [45]:
predict = model.predict_classes(x_test)

In [75]:
def test(num):
    print(int(np.dot(y_test[num], [0, 1, 2, 3, 4, 5, 6, 7])))
    print('神經網路判斷為:', predict[num])

In [76]:
interact_manual(test, num=(0, len(y_test)-1))

A Jupyter Widget

<function __main__.test>

## 4. 儲存model與權重

In [48]:
model_json = model.to_json()
open('my_model.json', 'w').write(model_json)
model.save_weights('my_model_weights.h5')

## 5. 修改模型再訓練一次

和上一個model的差別在於，這裡的LSTM層的神經元數目為原來的2倍

In [51]:
model2 = Sequential()
model2.add(Embedding(len(word_dict)+1, 256))
model2.add(LSTM(256))
model2.add(Dropout(0.3))
model2.add(Dense(y_train.shape[1]))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 256)         1667584   
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 2056      
_________________________________________________________________
activation_4 (Activation)    (None, 8)                 0         
Total params: 2,194,952
Trainable params: 2,194,952
Non-trainable params: 0
_________________________________________________________________


然而結果並沒有太大的差距，正確率仍在82%左右

In [52]:
model2.fit(x_train, y_train, batch_size=1000, epochs=5)

Epoch 1/5
341292/341292 [==============================] - 102s 300us/step - loss: 0.7848 - acc: 0.7129
Epoch 2/5
341292/341292 [==============================] - 99s 290us/step - loss: 0.5627 - acc: 0.7967
Epoch 3/5
341292/341292 [==============================] - 99s 291us/step - loss: 0.5324 - acc: 0.8053
Epoch 4/5
341292/341292 [==============================] - 99s 291us/step - loss: 0.5083 - acc: 0.8136
Epoch 5/5
341292/341292 [==============================] - 99s 291us/step - loss: 0.4876 - acc: 0.8205


In [53]:
score2 = model2.evaluate(x_test, y_test)

146269/146269 [==============================] - 56s 382us/step


測試資料的正確率依然在80%左右

In [54]:
print('loss:', score2[0])
print('正確率', score2[1])

loss: 0.534489005691
正確率 0.804538213836


## 6. 將結果進行動態呈現

In [55]:
predict2 = model2.predict_classes(x_test)

In [77]:
def test2(num):
    print(int(np.dot(y_test[num], [0, 1, 2, 3, 4, 5, 6, 7])))
    print('神經網路判斷為:', predict2[num])

In [78]:
interact_manual(test2, num=(0, len(y_test)-1))

A Jupyter Widget

<function __main__.test2>

## 7. 儲存model與權重

In [59]:
model_json2 = model2.to_json()
open('my_model2.json', 'w').write(model_json2)
model2.save_weights('my_model_weights2.h5')

## 8. 修改模型再訓練一次

再將LSTM層的神經元數目增加1倍

In [62]:
model3 = Sequential()
model3.add(Embedding(len(word_dict)+1, 256))
model3.add(LSTM(512))
model3.add(Dropout(0.3))
model3.add(Dense(y_train.shape[1]))
model3.add(Activation('softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 256)         1667584   
_________________________________________________________________
lstm_9 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 4104      
_________________________________________________________________
activation_6 (Activation)    (None, 8)                 0         
Total params: 3,246,600
Trainable params: 3,246,600
Non-trainable params: 0
_________________________________________________________________


準確率仍在82%

In [63]:
model3.fit(x_train, y_train, batch_size=1000, epochs=5)

Epoch 1/5
341292/341292 [==============================] - 256s 751us/step - loss: 0.7656 - acc: 0.7207
Epoch 2/5
341292/341292 [==============================] - 254s 744us/step - loss: 0.5620 - acc: 0.7959
Epoch 3/5
341292/341292 [==============================] - 253s 743us/step - loss: 0.5310 - acc: 0.8062
Epoch 4/5
341292/341292 [==============================] - 253s 743us/step - loss: 0.5072 - acc: 0.8134
Epoch 5/5
341292/341292 [==============================] - 255s 746us/step - loss: 0.4846 - acc: 0.8210


測試資料準確率也差不多是80%

In [64]:
score3 = model3.evaluate(x_test, y_test)
print('loss:', score3[0])
print('正確率', score3[1])

146269/146269 [==============================] - 79s 537us/step
loss: 0.529486226842
正確率 0.806131169289


## 9. 將結果進行動態呈現

In [65]:
predict3 = model3.predict_classes(x_test)

In [73]:
def test3(num):
    print(int(np.dot(y_test[num], [0, 1, 2, 3, 4, 5, 6, 7])))
    print('神經網路判斷為:', predict3[num])

In [74]:
interact_manual(test3, num=(0, len(y_test)-1))

A Jupyter Widget

<function __main__.test3>

## 10. 儲存model與權重

In [68]:
model_json3 = model3.to_json()
open('my_model3.json', 'w').write(model_json3)
model3.save_weights('my_model_weights3.h5')